In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
import time
import datetime

#HP Tuning
from tensorboard.plugins.hparams import api as hp

%load_ext tensorboard

In [2]:
InputData = np.loadtxt("Data/dataInputLarge.csv",delimiter=",")
OutputData = np.loadtxt ("Data/dataOutputLarge.csv", delimiter=",")

InputDataSmall = np.loadtxt("Data/dataInput2.csv",delimiter=",")
OutputDataSmall = np.loadtxt ("Data/dataOutput2.csv", delimiter=",")

InputDataLarge = np.loadtxt("Data/dataInputLarge2.csv",delimiter=",")
OutputDataLarge = np.loadtxt ("Data/dataOutputLarge2.csv", delimiter=",")



In [3]:
InputDataTrain = InputData[0:9000]  
InputDataTest = InputData[9000:11006]  
OutputDataTrain = OutputData[0:9000] 
OutputDataTest = OutputData[9000:11006]

InputDataTrainS = InputDataSmall[0:900]  
InputDataTestS = InputDataSmall[900:1106]  
OutputDataTrainS = OutputDataSmall[0:900] 
OutputDataTestS = OutputDataSmall[900:1106]

InputDataTrainL = InputDataLarge[0:18000]  
InputDataTestL = InputDataLarge[18000:22012]  
OutputDataTrainL = OutputDataLarge[0:18000] 
OutputDataTestL = OutputDataLarge[18000:22012]

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))

InputDataTrain_transformed = scaler.fit_transform(InputDataTrain) 
#InputDataTrain_retransformed = scaler.inverse_transform(InputDataTrain_transformed)
InputDataTest_transformed = scaler.fit_transform(InputDataTest)
#InputDataTest_retransformed = scaler.inverse_transform(InputDataTest_transformed)
OutputDataTrain_transformed = scaler.fit_transform(OutputDataTrain)
#OutputDataTrain_retransformed = scaler.inverse_transform(OutputDataTrain_transformed)
OutputDataTest_transformed = scaler.fit_transform(OutputDataTest)
#OutputDataTest_retransformed = scaler.inverse_transform(OutputDataTest_transformed)         


InputDataTrainS_transformed = scaler.fit_transform(InputDataTrainS) 
InputDataTestS_transformed = scaler.fit_transform(InputDataTestS)
OutputDataTrainS_transformed = scaler.fit_transform(OutputDataTrainS)
OutputDataTestS_transformed = scaler.fit_transform(OutputDataTestS)

InputDataTrainL_transformed = scaler.fit_transform(InputDataTrainL) 

InputDataTrainL_retransformed =  scaler.inverse_transform(InputDataTrainL_transformed)

InputDataTestL_transformed = scaler.fit_transform(InputDataTestL)
InputDataTestL_retransformed = scaler.inverse_transform(InputDataTestL_transformed)
OutputDataTrainL_transformed = scaler.fit_transform(OutputDataTrainL)
OutputDataTestL_transformed = scaler.fit_transform(OutputDataTestL)
OutputDataTestL_retransformed = scaler.inverse_transform(OutputDataTestL_transformed)  

In [5]:
def create_model() :
    return tf.keras.models.Sequential([
      tf.keras.layers.Dense(62,input_shape = (31,),activation='relu'),
      tf.keras.layers.Dense(62,activation="relu"),
      tf.keras.layers.Dense(31,activation="linear"),
    ])

In [3]:
#modelName ="Model"
#modelName ="ModelLarge"

model_load = tf.keras.models.load_model("Models/vscModel")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
#model = create_model()
model = model_load

In [8]:
model.summary()
model.output_shape
#model.layers[1].get_config()

Model: "sequential_171"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_536 (Dense)            (None, 6200)              198400    
_________________________________________________________________
dense_537 (Dense)            (None, 31)                192231    
Total params: 390,631
Trainable params: 390,631
Non-trainable params: 0
_________________________________________________________________


(None, 31)

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
model.fit(x=InputDataTrain_transformed, 
          y=OutputDataTrain_transformed, 
          epochs=200,
          batch_size = 200,
          validation_data=(InputDataTest_transformed, OutputDataTest_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [25]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)


63/63 [==============================] - 0s 560us/step - loss: 7.9384e-04 - accuracy: 0.8175
Test loss, Test acc: [0.0007938429480418563, 0.8175473809242249]


In [26]:
model.fit(x=InputDataTrainS_transformed, 
          y=OutputDataTrainS_transformed, 
          epochs=1000,
          batch_size = 200,
          validation_data=(InputDataTestS_transformed, OutputDataTestS_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [27]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)


63/63 [==============================] - 0s 587us/step - loss: 0.0012 - accuracy: 0.8036
Test loss, Test acc: [0.0012068396899849176, 0.8035892248153687]


In [28]:
model.fit(x=InputDataTrainL_transformed, 
          y=OutputDataTrainL_transformed, 
          epochs=1000,
          batch_size = 500,
          validation_data=(InputDataTestL_transformed, OutputDataTestL_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [29]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)



 1/63 [..............................] - ETA: 0s - loss: 1.5280e-04 - accuracy: 0.9688WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0000s vs `on_test_batch_end` time: 0.0010s). Check your callbacks.

63/63 [==============================] - 0s 510us/step - loss: 4.5774e-04 - accuracy: 0.8335
Test loss, Test acc: [0.00045773846795782447, 0.8334994912147522]


In [2]:
%tensorboard --logdir logs --host localhost --port 6003

Launching TensorBoard...
ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 9220.


In [13]:
%run Code.ipynb

In [21]:
InputDataTrainL_retransformed[2:3]

array([[ 12., -12.,  12., -12.,  12., -12.,  12., -12.,  12., -12.,  12.,
        -12.,  12., -12.,  12., -12.,  12., -12.,  12., -12.,  12., -12.,
         12., -12.,  12., -12.,  12., -12.,  12., -12.,  12.]])

In [15]:
first = 4011
second = 4012

#predictions = model.predict(InputDataTestL_transformed[first:second])
predictions = model.predict(InputDataTrainL_transformed[2:3])
print("predictions shape:", predictions.shape)
print("InputDataTest:",InputDataTestL_transformed[first:second])
print("InputDataTest in db (Command Gains)", InputDataTestL_retransformed[first:second])
print("OutputDataTest:",OutputDataTestL_transformed[first:second])
print("Predictions:",predictions)

print("OutputDataTest in dB:",scaler.inverse_transform(OutputDataTestL_transformed[first:second]))
print("Predictions in dB",scaler.inverse_transform(predictions))

print("Diff:", scaler.inverse_transform(OutputDataTestL_transformed[first:second])-scaler.inverse_transform(predictions))

predictions shape: (1, 31)
InputDataTest: [[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0.
  0. 1. 1. 1. 0. 0. 0.]]
InputDataTest in db (Command Gains) [[-12.  12.  12.  12. -12. -12. -12.  12.  12.  12. -12. -12. -12.  12.
   12.  12. -12. -12. -12.  12.  12.  12. -12. -12. -12.  12.  12.  12.
  -12. -12. -12.]]
OutputDataTest: [[0.0127431  0.79354449 0.61635357 0.79405423 0.20883644 0.38911241
  0.2044953  0.79246282 0.60984248 0.79299105 0.2103388  0.3869003
  0.20867506 0.79251361 0.61084695 0.79468082 0.20788832 0.38985948
  0.20435166 0.79233877 0.61484171 0.79121814 0.21151179 0.3912551
  0.20715985 0.78755473 0.62289564 0.81287893 0.23814399 0.36674687
  0.08429018]]
Predictions: [[0.9586915  0.02760983 0.95946157 0.05885313 0.9526511  0.05621229
  0.94541776 0.04809687 0.95185065 0.05539909 0.9514058  0.05245154
  0.94814587 0.05472778 0.9481592  0.05605498 0.94560224 0.05581859
  0.94828093 0.04969576 0.9516028  0.05403222 0.94670063 0.04863603
  0.95

In [16]:
#commandGains = InputDataTestL_retransformed[first:second].reshape((31,1)) 
commandGains = InputDataTrainL_retransformed[2:3].reshape((31,1)) 
#print(commandGains)
filterGainsPredicted = scaler.inverse_transform(predictions).reshape((31,1)) 
#print(filterGainsPredicted)

In [17]:
thirdOctaveGEQwithPredictions(commandGains,filterGainsPredicted)

C:/Users/Robert/AppData/Local/Temp/xpython_11260/3688153927.py:52: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_11260/3527972145.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*math.cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_11260/3527972145.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarra

In [15]:
#model.save("Models/"+modelName)